In [9]:

import csv
import msgpack
import pandas as pd
import numpy as np
import time
import argparse
import difflib
import json
import pickle
from collections import defaultdict
from collections import deque
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship, NodeMatch, NodeMatcher
import itertools

In [78]:
df = pd.read_csv('../data/data_product_202303280833.csv')

In [91]:
df['keys'] = ""

In [108]:
df['assetType'] = 'data product'

In [122]:
df.head()

,data_product_id,name,tags,product_metadata,keys,assetType
0,e73e30222cd0ed412a79a0804ecd38f1,test1,NaN,"{""metadata"":[{""domain_id"":4,""fieldName"":""terri...","{""fieldNames"": [""territoryID"", ""region"", ""coun...",data product
1,7b1d9bce9bffcbb8c62c8fb4b3dea2da,DP1 13_03,"[""TEST""]","{""metadata"":[{""domain_id"":89,""fieldName"":""c_cu...","{""fieldNames"": [""c_custkey"", ""c_mktsegment""], ...",data product
2,a4e19f89bdc9b96d0867713591631dc6,test-dataproduct001,"[""Test""]","{""metadata"":[]}","{""fieldNames"": [], ""businessNames"": [], ""busin...",data product
3,073afc4d623905096dc093732a050442,test-data-01,"[""Test"", ""NEW""]","{""metadata"":[{""domain_id"":0,""fieldName"":""O_ORD...","{""fieldNames"": [""O_ORDERSTATUS"", ""O_ORDERKEY""]...",data product
4,63456ab82912afd85fa51c13f5fcd8ec,test da,"[""Test""]","{""metadata"":[]}","{""fieldNames"": [], ""businessNames"": [], ""busin...",data product


In [81]:
df.loc[0, 'keys'] = json.dumps({"test": [4, 5, 6], "next": [6, 8, 9]})

In [123]:
x1 = json.dumps({"test": [4, 5, 6], "next": [6, 8, 9], "keys" : {"new": [1, 2, 3], "again": [4, 5, 6]}})
y1 = json.loads(x1)
y1

{'test': [4, 5, 6],
 'next': [6, 8, 9],
 'keys': {'new': [1, 2, 3], 'again': [4, 5, 6]}}

In [113]:
x = str(df.loc[0, 'keys'])
y = json.loads(x)

In [172]:
y

{'fieldNames': ['territoryID', 'region', 'country'],
 'businessNames': ['territoryID', 'region', 'country'],
 'businessDescriptions': ['territoryID', 'region', 'country']}

In [18]:
testRow = json.loads(df['product_metadata'][0])

In [37]:
testRow['metadata'][2]

{'domain_id': 5,
 'fieldName': 'country',
 'businessName': 'country',
 'businessDescription': 'country',
 'dataType': 'string',
 'schema': 'northwind',
 'table': 'employees',
 'id': 3,
 'recommendedSensitivity': 'public',
 'dataDomain': 'Coordinates',
 'recomendedSensitivity': 'public'}

In [42]:
def get_rows_inner(df):
    rows = [json.loads(row) for row in df['product_metadata']]
    return rows

def extract_metadata_inner(df):
    rows = get_rows_inner(df)
    fieldNames, businessNames, businessDescriptions = [], [], []
    for row in rows:
        innerField, innerName, innerDesc = [], [], []
        for idx in row['metadata']:
            innerField.append(idx['fieldName'])
            innerName.append(idx['businessName'])
            innerDesc.append(idx['businessDescription'])
        fieldNames.append(innerField)
        businessNames.append(innerName)
        businessDescriptions.append(innerDesc)
    return fieldNames, businessNames, businessDescriptions

def add_cols_inner(df):
    fieldNames, businessNames, businessDescriptions = extract_metadata_inner(df)
    df = df.assign(fieldNames=fieldNames, businessNames=businessNames, businessDescriptions=businessDescriptions)
    return df

In [90]:
def extract_metadata_inner_neo(df):
    rows = get_rows_inner(df)
    fieldNames, businessNames, businessDescriptions = [], [], []
    rowCnt = 0
    for row in rows:
        innerField, innerName, innerDesc = [], [], []
        for idx in row['metadata']:
            innerField.append(idx['fieldName'])
            innerName.append(idx['businessName'])
            innerDesc.append(idx['businessDescription'])
        keys = json.dumps({"fieldNames": innerField, "businessNames": innerName, "businessDescriptions": innerDesc})
        df.loc[rowCnt, 'keys'] = keys
        rowCnt += 1

In [94]:
extract_metadata_inner_neo(df)

In [124]:
df.head()

,data_product_id,name,tags,product_metadata,keys,assetType
0,e73e30222cd0ed412a79a0804ecd38f1,test1,NaN,"{""metadata"":[{""domain_id"":4,""fieldName"":""terri...","{""fieldNames"": [""territoryID"", ""region"", ""coun...",data product
1,7b1d9bce9bffcbb8c62c8fb4b3dea2da,DP1 13_03,"[""TEST""]","{""metadata"":[{""domain_id"":89,""fieldName"":""c_cu...","{""fieldNames"": [""c_custkey"", ""c_mktsegment""], ...",data product
2,a4e19f89bdc9b96d0867713591631dc6,test-dataproduct001,"[""Test""]","{""metadata"":[]}","{""fieldNames"": [], ""businessNames"": [], ""busin...",data product
3,073afc4d623905096dc093732a050442,test-data-01,"[""Test"", ""NEW""]","{""metadata"":[{""domain_id"":0,""fieldName"":""O_ORD...","{""fieldNames"": [""O_ORDERSTATUS"", ""O_ORDERKEY""]...",data product
4,63456ab82912afd85fa51c13f5fcd8ec,test da,"[""Test""]","{""metadata"":[]}","{""fieldNames"": [], ""businessNames"": [], ""busin...",data product


In [125]:
df.iloc[0]['keys']

'{"fieldNames": ["territoryID", "region", "country"], "businessNames": ["territoryID", "region", "country"], "businessDescriptions": ["territoryID", "region", "country"]}'

In [126]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"

In [127]:
graph = Graph(uri, auth=(username, password))

In [129]:
def create_nodes_neo(df, graph):
    root_node = Node("global_search", id=-1)
    existing_node = graph.nodes.match("global_search", id=-1).first()
    if existing_node:
        root_node = existing_node
    else:
        graph.create(root_node)

    for index, row in df.iterrows():
        # create node
        node_props = {
            "id": row["data_product_id"],
            "name": row["name"],
            "assetType": row['assetType'],
            "keys": row["keys"]
        }
        node = Node("global_search", **node_props)
        graph.create(node)
        rel = Relationship(root_node, "HAS", node)
        graph.create(rel)

In [130]:
create_nodes_neo(df, graph)

In [161]:
def insert_node(object, graph):
    extracted = json.loads(object)
    root_node = Node("global_search", id=-1)

    existing_root = graph.nodes.match("global_search", id=-1).first()
    if existing_root:
        root_node = existing_root
    else:
        graph.create(root_node)

    node_props = {
        "id": extracted['id'],
        'name': extracted['name'],
        'assetType': extracted['assetType'],
        'keys': str(extracted['keys'])
    }

    node = Node("global_search", **node_props)

    existing_node = graph.nodes.match("global_search", id=extracted['id'], assetType=extracted['assetType']).first()
    if existing_node:
        existing_node.update(node_props)
        graph.push(existing_node)
        return {"EXISTS": f"updated node with id {extracted['id']} and asset type {extracted['assetType']}"}
    else:
        graph.create(node)
        rel = Relationship(root_node, "HAS", node)
        graph.create(rel)
        return {"NEW NODE": f"inserted node with id {extracted['id']} and asset type {extracted['assetType']}"}

In [176]:
test = json.dumps({"id": "0", "name": "RAKESH_N", "assetType": "TEST", "keys" : {"first": ["1", "2", "3"], "second": ["4", "5", "6"]}})
newTest = json.dumps({"id": "0", "name": "CHANGE_RAKESH", "assetType": "TEST", "keys" : {"newFirst": ["2", "3", "4"], "newSecond": ["5", "6", "7"]}})

In [185]:
insert_node(test, graph)

{'NEW NODE': 'inserted node with id 0 and asset type TEST'}

In [166]:
insert_node(newTest, graph)

{'EXISTS': 'updated node with id 0 and asset type TEST'}

In [163]:
def delete_node(id, assetType, graph):
    node = graph.nodes.match("global_search", id=id, assetType=assetType).first()
    if node:
        graph.delete(node)
        return {"DELETE": f"Deleted node with id {id} and assetType {assetType}"}
    else:
        return {"ERR": f"No node with id {id} and assetType {assetType} found"}

In [183]:
delete_node('0', 'TEST', graph)

{'DELETE': 'Deleted node with id 0 and assetType TEST'}

In [180]:
def fuzzy_full_text_search(word, n):
    query = """
        CALL db.index.fulltext.queryNodes("globalSearchNeo", $word, {limit:$n}) YIELD node, score
        RETURN node.id, node.name, node.assetType, node.keys, score
    """
    parameters = {"word": word + "~", "n": n}
    return graph.run(query, parameters=parameters).data()


In [187]:
fuzzy_full_text_search("territory", 10)

[{'node.id': 'c254cd78ea01f51ebaaceba6d738750f',
  'node.name': nan,
  'node.assetType': 'data product',
  'node.keys': '{"fieldNames": ["territoryID", "region", "country"], "businessNames": ["territoryID", "region", "country"], "businessDescriptions": ["territoryID", "region", "country"]}',
  'score': 2.615947723388672},
 {'node.id': 'e73e30222cd0ed412a79a0804ecd38f1',
  'node.name': 'testDemo',
  'node.assetType': 'data product',
  'node.keys': "{'fieldNames': ['territoryID', 'region', 'country'], 'businessNames': ['territoryID', 'region', 'country'], 'businessDescriptions': ['territoryID', 'region', 'country']}",
  'score': 2.615947723388672},
 {'node.id': '38db7a16b8415a6afea41e185736b0d1',
  'node.name': 'DP 2',
  'node.assetType': 'data product',
  'node.keys': '{"fieldNames": ["territoryID", "region", "country"], "businessNames": ["territoryID", "region", "country"], "businessDescriptions": ["territoryID", "region", "country"]}',
  'score': 2.615947723388672}]

In [31]:
def update_description(word, new_description):
    query = "MATCH (n:business_modeler_terms {name: $word}) SET n.description = $new_description"
    graph.run(query, word=word, new_description=new_description)

In [32]:
update_description("test", "TEST NEW DESC")

In [29]:
def insert_node(word, description):
    node = Node("business_modeler_terms", name=word, description=description)
    graph.create(node)

    root_node = graph.nodes.match("business_modeler_terms", name="", description="").first()
    relationship = Relationship(root_node, "HAS", node)

    graph.create(relationship)

In [76]:
insert_node("CAPTEST", "capital")